In [59]:
import pandas as pd
import numpy as np
from scipy.stats import t as t_dist, norm
from scipy.linalg import cholesky
import warnings

portfolio_df = pd.read_csv('portfolio.csv')
prices_df = pd.read_csv('DailyPrices.csv')
prices_df['Date'] = pd.to_datetime(prices_df['Date'])

In [70]:
# Daily Returns
def return_calculate(prices: pd.DataFrame, method="DISCRETE", date_column="Date"):
    if date_column not in prices.columns:
        raise ValueError(f"date_column: {date_column} not in DataFrame")

    vars = prices.columns.tolist()
    vars.remove(date_column)

    p = prices[vars].to_numpy()
    n, m = p.shape
    p2 = np.zeros((n-1, m))

    if method.upper() == "DISCRETE":
        p2 = (p[1:, :] / p[:-1, :]) - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p[1:, :] / p[:-1, :])
    else:
        raise ValueError(f"Invalid method '{method}'. Must be 'DISCRETE' or 'LOG'.")

    dates = prices[date_column].iloc[1:].reset_index(drop=True)
    returns_df = pd.DataFrame(p2, columns=vars)
    returns_df.insert(0, date_column, dates)
    
    return returns_df

daily_returns = return_calculate(prices_df, method="DISCRETE")

In [71]:
# Initialize Portfolio Value
def initialize_portfolio(portfolio):
    portfolio['CurrentValue'] = portfolio['Holding'] * portfolio['Starting Price']
    return portfolio

# Fit Distributions and Transform to Uniform
def fit_and_transform(portfolio, returns):
    models = {}
    uniform = pd.DataFrame()
    standard_normal = pd.DataFrame()

    for stock in portfolio["Stock"]:
        dist_type = portfolio.loc[portfolio['Stock'] == stock, 'Distribution'].iloc[0]
        
        if dist_type == 'Normal': 
            mu, sigma = norm.fit(returns[stock])
            models[stock] = (mu, sigma)
            uniform[stock] = norm.cdf(returns[stock], loc=mu, scale=sigma)
            standard_normal[stock] = norm.ppf(uniform[stock])
        
        elif dist_type == 'T':
            nu, mu, sigma = t_dist.fit(returns[stock])
            models[stock] = (nu, mu, sigma)
            uniform[stock] = t_dist.cdf(returns[stock], df=nu, loc=mu, scale=sigma)
            standard_normal[stock] = norm.ppf(uniform[stock])
    
    return models, standard_normal

In [72]:
# Simulate Copula
def simulate_copula(standard_normal, nSim):
    spearman_corr_matrix = standard_normal.corr(method='spearman')
    cholesky_decomp = cholesky(spearman_corr_matrix, lower=True)
    
    random_normals = np.random.normal(size=(nSim, standard_normal.shape[1]))
    correlated_samples = random_normals @ cholesky_decomp.T
    uni = pd.DataFrame(norm.cdf(correlated_samples), columns=standard_normal.columns)
    
    return uni

def generate_simulated_returns(portfolio, uni, models):
    simulated_returns = pd.DataFrame()
    for stock in portfolio["Stock"]:
        dist_type = portfolio.loc[portfolio['Stock'] == stock, 'Distribution'].iloc[0]
        
        if dist_type == 'Normal':
            mu, sigma = models[stock]
            simulated_returns[stock] = norm.ppf(uni[stock], loc=mu, scale=sigma)
            
        elif dist_type == 'T':
            nu, mu, sigma = models[stock]
            simulated_returns[stock] = t_dist.ppf(uni[stock], df=nu, loc=mu, scale=sigma)
    
    return simulated_returns

In [76]:
# Calculate Simulated PnLs
def calculate_simulated_values(portfolio, simulated_returns):
    simulated_value = pd.DataFrame()
    pnl = pd.DataFrame()
    
    for stock in portfolio["Stock"]:
        current_value = portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0]
        simulated_value[stock] = current_value * (1 + simulated_returns[stock])
        pnl[stock] = simulated_value[stock] - current_value
    
    return pnl

# VaR and ES
def calculate_risk_metrics(portfolio, pnl):
    risk = pd.DataFrame(columns=["Stock", "VaR95", "ES95", "VaR95_Pct", "ES95_Pct"])
    weights = pd.DataFrame()
    
    for stock in pnl.columns:
        i = risk.shape[0]
        risk.loc[i, "Stock"] = stock
        risk.loc[i, "VaR95"] = -np.percentile(pnl[stock], 5)
        risk.loc[i, "VaR95_Pct"] = risk.loc[i, "VaR95"] / portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0]
        risk.loc[i, "ES95"] = -pnl[stock][pnl[stock] <= -risk.loc[i, "VaR95"]].mean()
        risk.loc[i, "ES95_Pct"] = risk.loc[i, "ES95"] / portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0]
        
        weights.at['Weight', stock] = portfolio.loc[portfolio['Stock'] == stock, 'CurrentValue'].iloc[0] / portfolio['CurrentValue'].sum()
    
    pnl['Total'] = pnl.sum(axis=1)
    i = risk.shape[0]
    risk.loc[i, "Stock"] = 'Total'
    risk.loc[i, "VaR95"] = -np.percentile(pnl['Total'], 5)
    risk.loc[i, "VaR95_Pct"] = risk.loc[i, "VaR95"] / portfolio['CurrentValue'].sum()
    risk.loc[i, "ES95"] = -pnl['Total'][pnl['Total'] <= -risk.loc[i, "VaR95"]].mean()
    risk.loc[i, "ES95_Pct"] = risk.loc[i, "ES95"] / portfolio['CurrentValue'].sum()
    
    return risk

In [77]:
def simulateCopula(portfolio, returns):
    portfolio = initialize_portfolio(portfolio)
    models, standard_normal = fit_and_transform(portfolio, returns)
    uni = simulate_copula(standard_normal, nSim=10000)
    simulated_returns = generate_simulated_returns(portfolio, uni, models)
    pnl = calculate_simulated_values(portfolio, simulated_returns)
    risk = calculate_risk_metrics(portfolio, pnl)
    return risk

warnings.filterwarnings("ignore")
portfolios = ["A", "B", "C"]

def display_risk_metrics(label, risk_data):
    total_line = risk_data[risk_data.iloc[:, 0] == "Total"]
    var_value = total_line.iloc[0, 1]
    es_value = total_line.iloc[0, 2]
    var_dec = total_line.iloc[0, 3]
    es_dec = total_line.iloc[0, 4]
    
    print(f"Portfolio {label} VaR: ${var_value:.2f} and VaR_decimal: {var_dec:.5f}")
    print(f"Portfolio {label} ES:  ${es_value:.2f} and ES_decimal: {es_dec:.5f}")
    print()

In [78]:
portfolio_df.loc[portfolio_df['Portfolio'].isin(['A', 'B']), 'Distribution'] = 'T'
portfolio_df.loc[portfolio_df['Portfolio'] == 'C', 'Distribution'] = 'Normal'

for stock in portfolio_df["Stock"]:
    portfolio_df.loc[portfolio_df['Stock'] == stock, 'Starting Price'] = prices_df.iloc[-1][stock]

for label in portfolios:
    each_port = portfolio_df.loc[portfolio_df["Portfolio"] == label]
    risk = simulateCopula(each_port, daily_returns)
    display_risk_metrics(label, risk)

total_risk = simulateCopula(portfolio_df, daily_returns)
display_risk_metrics("Total", total_risk)


Portfolio A VaR: $4779.29 and VaR_decimal: 0.01263
Portfolio A ES:  $6653.99 and ES_decimal: 0.01758

Portfolio B VaR: $4010.80 and VaR_decimal: 0.01050
Portfolio B ES:  $5815.93 and ES_decimal: 0.01523

Portfolio C VaR: $3579.08 and VaR_decimal: 0.01176
Portfolio C ES:  $4527.63 and ES_decimal: 0.01488

Portfolio Total VaR: $12013.33 and VaR_decimal: 0.01128
Portfolio Total ES:  $16192.12 and ES_decimal: 0.01521

